# snehal Jagtap

### Recommendation System

### Step 1 :Import libraries

In [2]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

In [3]:
data = pd.read_csv('anime.csv')

### Step 2 :Preprocessing

In [4]:
data.shape

(12294, 7)

In [5]:
data.head()


,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [6]:
data.tail()

,anime_id,name,genre,type,episodes,rating,members
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175
12293,26081,Yasuji no Pornorama: Yacchimae!!,Hentai,Movie,1,5.46,142


In [7]:
data.describe()

,anime_id,rating,members
count,12294.000000,12064.000000,1.229400e+04
mean,14058.221653,6.473902,1.807134e+04
std,11455.294701,1.026746,5.482068e+04
min,1.000000,1.670000,5.000000e+00
25%,3484.250000,5.880000,2.250000e+02
50%,10260.500000,6.570000,1.550000e+03
75%,24794.500000,7.180000,9.437000e+03
max,34527.000000,10.000000,1.013917e+06


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


In [9]:
data.isnull().sum()

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

In [10]:
data.dtypes

anime_id      int64
name         object
genre        object
type         object
episodes     object
rating      float64
members       int64
dtype: object

In [11]:
data['episodes'].unique()

array(['1', '64', '51', '24', '10', '148', '110', '13', '201', '25', '22',
       '75', '4', '26', '12', '27', '43', '74', '37', '2', '11', '99',
       'Unknown', '39', '101', '47', '50', '62', '33', '112', '23', '3',
       '94', '6', '8', '14', '7', '40', '15', '203', '77', '291', '120',
       '102', '96', '38', '79', '175', '103', '70', '153', '45', '5',
       '21', '63', '52', '28', '145', '36', '69', '60', '178', '114',
       '35', '61', '34', '109', '20', '9', '49', '366', '97', '48', '78',
       '358', '155', '104', '113', '54', '167', '161', '42', '142', '31',
       '373', '220', '46', '195', '17', '1787', '73', '147', '127', '16',
       '19', '98', '150', '76', '53', '124', '29', '115', '224', '44',
       '58', '93', '154', '92', '67', '172', '86', '30', '276', '59',
       '72', '330', '41', '105', '128', '137', '56', '55', '65', '243',
       '193', '18', '191', '180', '91', '192', '66', '182', '32', '164',
       '100', '296', '694', '95', '68', '117', '151', '130',

In [12]:
data['rating'].unique()

array([ 9.37,  9.26,  9.25,  9.17,  9.16,  9.15,  9.13,  9.11,  9.1 ,
        9.06,  9.05,  9.04,  8.98,  8.93,  8.92,  8.88,  8.84,  8.83,
        8.82,  8.81,  8.8 ,  8.78,  8.77,  8.76,  8.75,  8.74,  8.73,
        8.72,  8.71,  8.69,  8.68,  8.67,  8.66,  8.65,  8.64,  8.62,
        8.61,  8.6 ,  8.59,  8.58,  8.57,  8.56,  8.55,  8.54,  8.53,
        8.52,  8.51,  8.5 ,  8.49,  8.48,  8.47,  8.46,  8.45,  8.44,
        8.43,  8.42,  8.41,  8.4 ,  8.39,  8.38,  8.37,  8.36,  8.35,
        8.34,  8.33,  8.32,  8.31,  8.3 ,  8.29,  8.28,  8.27,  8.26,
        8.25,  8.24,  8.23,  8.22,  8.21,  8.2 ,  8.19,  8.18,  8.17,
        8.16,  8.15,  8.14,  8.13,  8.12,  8.11,  8.1 ,  8.09,  8.08,
        8.07,  8.06,  8.05,  8.04,  8.03,  8.02,  8.01,  8.  ,  7.99,
        7.98,  7.97,  7.96,  7.95,  7.94,  7.93,  7.92,  7.91,  7.9 ,
        7.89,  7.88,  7.87,  7.86,  7.85,  7.84,  7.83,  7.82,  7.81,
        7.8 ,  7.79,  7.78,  7.77,  7.76,  7.75,  7.74,  7.73,  7.72,
        7.71,  7.7 ,

In [14]:
data['episodes'] = pd.to_numeric(data['episodes'], errors='coerce')
data['episodes'].fillna(data['episodes'].mean(), inplace=True)

In [15]:
data['rating'] = pd.to_numeric(data['rating'], errors='coerce')
data['rating'].fillna(data['rating'].mean(), inplace=True)

In [16]:
# Normalize the ratings column
scaler = MinMaxScaler()
data['rating_normalized'] = scaler.fit_transform(data[['rating']])

In [17]:
data['genre'].fillna('Unknown', inplace=True)

In [18]:
data.isnull().sum()

anime_id              0
name                  0
genre                 0
type                 25
episodes              0
rating                0
members               0
rating_normalized     0
dtype: int64

In [19]:
# Combine relevant features into a single string for vectorization (e.g., genres, title)
if 'name' in data.columns:
    data['combined_features'] = data['name'] + ' ' + data['genre']
elif 'anime_title' in data.columns:
    data['combined_features'] = data['anime_title'] + ' ' + data['genre']
else:
    print("Title column not found. Please verify the dataset structure.")


In [20]:
data.head()

,anime_id,name,genre,type,episodes,rating,members,rating_normalized,combined_features
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1.0,9.37,200630,0.924370,"Kimi no Na wa. Drama, Romance, School, Superna..."
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64.0,9.26,793665,0.911164,"Fullmetal Alchemist: Brotherhood Action, Adven..."
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51.0,9.25,114262,0.909964,"Gintama° Action, Comedy, Historical, Parody, S..."
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24.0,9.17,673572,0.900360,"Steins;Gate Sci-Fi, Thriller"
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51.0,9.16,151266,0.899160,"Gintama&#039; Action, Comedy, Historical, Paro..."


In [21]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['combined_features'])

### Step 3: Recommendation System

In [22]:
# Calculate cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [23]:
# Function to recommend anime based on cosine similarity
def recommend_anime(title, cosine_sim=cosine_sim, df=data):
    # Check which column contains the anime title
    title_column = 'title'  # Update this to the correct column name if needed

    if 'name' in df.columns:
        title_column = 'name'
    elif 'anime_title' in df.columns:
        title_column = 'anime_title'

    # Get the index of the anime that matches the title
    try:
        idx = df[df[title_column] == title].index[0]
    except IndexError:
        return f"Anime titled '{title}' not found in the dataset."

    # Get the pairwise similarity scores of all anime with the given anime
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the anime based on similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the indices of the top 10 most similar anime (excluding itself)
    sim_scores = sim_scores[1:11]

    # Get the anime indices
    anime_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar anime
    return df[title_column].iloc[anime_indices]

In [24]:
recommend_anime('Naruto')

719                            The Last: Naruto the Movie
615                                    Naruto: Shippuuden
1343                                          Naruto x UT
486                              Boruto: Naruto the Movie
2458                 Naruto Shippuuden: Sunny Side Battle
784            Naruto: Shippuuden Movie 6 - Road to Ninja
1103    Boruto: Naruto the Movie - Naruto ga Hokage ni...
1472          Naruto: Shippuuden Movie 4 - The Lost Tower
2416    Naruto: Honoo no Chuunin Shiken! Naruto vs. Ko...
2997    Naruto Soyokazeden Movie: Naruto to Mashin to ...
Name: name, dtype: object

### Step 4: Evaluation

In [25]:
# Create a User-Anime Interaction Data
data['user_id'] = np.random.randint(1, 50, size=len(data))  
data['anime_id'] = data.index  
data['rating'] = pd.to_numeric(data['rating'], errors='coerce')

In [26]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [27]:
tfidf_matrix_train = tfidf.fit_transform(train_data['combined_features'])
cosine_sim_train = cosine_similarity(tfidf_matrix_train, tfidf_matrix_train)

In [28]:
def recommend_anime_from_train(title, df=train_data, cosine_sim=cosine_sim_train):
    title_column = 'name'  # Replace with correct column name if needed
    if 'name' in df.columns:
        title_column = 'name'
    elif 'anime_title' in df.columns:
        title_column = 'anime_title'

    if title not in df[title_column].values:
        return []

    idx = df[df[title_column] == title].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    anime_indices = [i[0] for i in sim_scores]
    return df[title_column].iloc[anime_indices].tolist()

In [29]:
# Evaluate the recommendation system
from sklearn.metrics import precision_score, recall_score, f1_score

In [30]:
y_true, y_pred = [], []

In [31]:
for _, row in test_data.iterrows():
    recommended_anime = recommend_anime_from_train(row['name'])
    y_true.append(1 if row['rating'] >= 3.5 else 0)
    y_pred.append(1 if row['name'] in recommended_anime else 0)

In [34]:
# Calculate Precision, Recall, and F1-Score
if len(y_true) == len(y_pred):
    # Calculate Precision, Recall, and F1-Score
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-Score: {f1:.2f}")
else:
    print("Error: Mismatch in the lengths of y_true and y_pred. Please check the data.")

Precision: 0.00
Recall: 0.00
F1-Score: 0.00


C:\Users\sneha\Downloads\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Interview Questions

**1. Can you explain the difference between user-based and item-based collaborative filtering?**


User-based collaborative filtering recommends items by finding users with similar preferences and suggesting items those similar users liked, while item-based collaborative filtering focuses on finding items similar to those a user has interacted with and recommending them. User-based filtering captures diverse preferences but struggles with scalability and data sparsity, whereas item-based filtering is more scalable and stable since item similarities are precomputed but may lack diversity in recommendations.

**2. What is collaborative filtering, and how does it work?**

Collaborative filtering is a recommendation technique that predicts a user’s preferences by analyzing patterns of shared behavior among users or items. It works by collecting user-item interaction data (e.g., ratings or purchases), identifying similarities between users or items, and recommending items based on these patterns. There are two main types: user-based filtering, which finds similar users, and item-based filtering, which finds similar items. While collaborative filtering provides personalized recommendations without domain knowledge, it faces challenges like the cold start problem, data sparsity, and scalability issues.